# Char level Seq2Seq example in Keras

modified from https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html


In [1]:
import re
from keras.models import Model , load_model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import ModelCheckpoint, Callback, ReduceLROnPlateau
import numpy as np

Using TensorFlow backend.


### Set up the network parameters

In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 130000  # Number of samples to train on.

### Get the data

In [3]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

Read in the doc and then split into lines

For each line split based on tabs

In [4]:
lines = [l.strip().split("  ") for l in open("./cmudict-0.7b", encoding='latin1') 
         if re.match('^[A-Z]', l)]
lines = [(w, ps.split()) for w, ps in lines]


In [5]:
phonemes = ["_"] + sorted(set(p for w, ps in lines for p in ps))
phonemes[:5]

['_', 'AA0', 'AA1', 'AA2', 'AE0']

In [6]:
lines = lines[:130000]
len(lines)
lines[100], lines[-1]

(("ABC'S", ['EY1', 'B', 'IY2', 'S', 'IY2', 'Z']),
 ('WHITES', ['W', 'AY1', 'T', 'S']))

In [7]:
for line in lines:
    #print(line[0])
    input_texts.append(line[0])
    for char in line[0]:
        if char not in input_characters:
            input_characters.add(char)
    target_text =['\t']
    target_text = target_text + line[1] + ['\n']
    target_texts.append(target_text)
    for char in line[1]:
        if char not in target_characters:
            target_characters.add(char)

In [8]:
for line in lines:
    #print(line[0])
    input_texts.append(line[0])
    for char in line[0]:
        if char not in input_characters:
            input_characters.add(char)
    target_text =['\t']
    target_text = target_text + line[1] + ['\n']
    target_texts.append(target_text)
    for char in line[1]:
        if char not in target_characters:
            target_characters.add(char)

In [9]:
type(target_characters)

set

In [10]:
target_characters.add('\t')
target_characters.add('\n')

In [11]:
print(len(input_texts))
print(len(target_texts))
print(input_texts[100:105])
print(target_texts[100:105])

260000
260000
["ABC'S", 'ABCO', 'ABCOTEK', 'ABCS', 'ABDALLA']
[['\t', 'EY1', 'B', 'IY2', 'S', 'IY2', 'Z', '\n'], ['\t', 'AE1', 'B', 'K', 'OW0', '\n'], ['\t', 'AE1', 'B', 'K', 'OW0', 'T', 'EH2', 'K', '\n'], ['\t', 'EY1', 'B', 'IY2', 'S', 'IY2', 'Z', '\n'], ['\t', 'AE2', 'B', 'D', 'AE1', 'L', 'AH0', '\n']]


In [12]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 260000
Number of unique input tokens: 44
Number of unique output tokens: 71
Max sequence length for inputs: 34
Max sequence length for outputs: 34


In [13]:
input_characters

["'",
 '(',
 ')',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '_',
 'À',
 'É']

In [14]:
target_characters

['\t',
 '\n',
 'AA0',
 'AA1',
 'AA2',
 'AE0',
 'AE1',
 'AE2',
 'AH0',
 'AH1',
 'AH2',
 'AO0',
 'AO1',
 'AO2',
 'AW0',
 'AW1',
 'AW2',
 'AY0',
 'AY1',
 'AY2',
 'B',
 'CH',
 'D',
 'DH',
 'EH0',
 'EH1',
 'EH2',
 'ER0',
 'ER1',
 'ER2',
 'EY0',
 'EY1',
 'EY2',
 'F',
 'G',
 'HH',
 'IH0',
 'IH1',
 'IH2',
 'IY0',
 'IY1',
 'IY2',
 'JH',
 'K',
 'L',
 'M',
 'N',
 'NG',
 'OW0',
 'OW1',
 'OW2',
 'OY0',
 'OY1',
 'OY2',
 'P',
 'R',
 'S',
 'SH',
 'T',
 'TH',
 'UH0',
 'UH1',
 'UH2',
 'UW0',
 'UW1',
 'UW2',
 'V',
 'W',
 'Y',
 'Z',
 'ZH']

### Make some look up tables

In [15]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [16]:
print(input_token_index['A'])
print(target_token_index['EY1'])

15
31


#### Make the matrix holders in correct shapes

In [17]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

In [18]:
print('Encoder Inputs:',encoder_input_data.shape)
print('Decoder Inputs:',decoder_input_data.shape)
print('Decoder Target:',decoder_target_data.shape)
print(encoder_input_data[0])

Encoder Inputs: (260000, 34, 44)
Decoder Inputs: (260000, 34, 71)
Decoder Target: (260000, 34, 71)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


#### Create the Sequences at a char level as OHE

In [19]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [20]:
print(input_texts[0])
print(encoder_input_data[0][0])
print(input_token_index['A'])
print(np.argmax(encoder_input_data[0][0]))

A
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]
15
15


### Make the model

In [21]:
# print(num_encoder_tokens)
print(latent_dim)
print(num_decoder_tokens)

256
71


In [22]:
reduce_LR = ReduceLROnPlateau(monitor='val_loss',factor = 0.9, patience=3,cooldown=2, min_lr = 0.00001)

#### The Encoder

In [23]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens)) 
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

#### The Decoder

In [24]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens),name='decoder_inputs')

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,name='LSTM_01') 
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [25]:
# Define the model that will turn `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [26]:
# compile the model to predict classes
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [27]:
# Run training
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=100,
          callbacks=[reduce_LR],
          validation_split=0.2)

Train on 208000 samples, validate on 52000 samples
Epoch 1/100
208000/208000 [==============================] - 160s - loss: 0.4703 - val_loss: 0.3894
Epoch 2/100
208000/208000 [==============================] - 159s - loss: 0.2695 - val_loss: 0.2078
Epoch 3/100
208000/208000 [==============================] - 159s - loss: 0.1577 - val_loss: 0.1362
Epoch 4/100
208000/208000 [==============================] - 158s - loss: 0.1135 - val_loss: 0.1063
Epoch 5/100
208000/208000 [==============================] - 158s - loss: 0.0910 - val_loss: 0.0888
Epoch 6/100
208000/208000 [==============================] - 158s - loss: 0.0771 - val_loss: 0.0780
Epoch 7/100
208000/208000 [==============================] - 158s - loss: 0.0673 - val_loss: 0.0717
Epoch 8/100
208000/208000 [==============================] - 158s - loss: 0.0597 - val_loss: 0.0641
Epoch 9/100
208000/208000 [==============================] - 159s - loss: 0.0536 - val_loss: 0.0591
Epoch 10/100
208000/208000 [=====================

In [28]:
# Save model
model.save('s2s-cmu-130k-gpu.h5')

In [29]:
#model = load_model('s2s-cmu.h5')

# Making predictions
 1) encode input and retrieve initial state for decoder  
 2) run one step of decoder with this initial state and pass in a "start of sequence" (\t) token as target. Output will be the next target token  
 3) Repeat with the next predicted token target token and current states

In [30]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

In [31]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

In [32]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [33]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [34]:
reverse_input_char_index[1]


'('

In [35]:
reverse_target_char_index[0]

'\t'

In [36]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    
    # Create and pass in the firsat token to start the predictions when combined with the decoeder state.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False #our stop condition will be '/n' which acts as <EOS>
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += "-" + sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [37]:
for seq_index in range(1000):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: A
Decoded sentence: -AH0-

-
Input sentence: A(1)
Decoded sentence: -EY1-

-
Input sentence: A'S
Decoded sentence: -EY1-Z-

-
Input sentence: A.
Decoded sentence: -EY1-

-
Input sentence: A.'S
Decoded sentence: -EY1-Z-

-
Input sentence: A.S
Decoded sentence: -EY1-Z-

-
Input sentence: A42128
Decoded sentence: -EY1-F-AO1-R-T-UW1-W-AH1-N-T-UW1-EY1
-
Input sentence: AA
Decoded sentence: -EY2-EY1-

-
Input sentence: AAA
Decoded sentence: -T-R-IH2-P-AH0-L-EY1-

-
Input sentence: AABERG
Decoded sentence: -AA1-B-ER0-G-

-
Input sentence: AACHEN
Decoded sentence: -AA1-K-AH0-N-

-
Input sentence: AACHENER
Decoded sentence: -AA1-K-AH0-N-ER0-

-
Input sentence: AAH
Decoded sentence: -AA1-

-
Input sentence: AAKER
Decoded sentence: -AA1-K-ER0-

-
Input sentence: AALIYAH
Decoded sentence: -AA2-L-IY1-AA2-

-
Input sentence: AALSETH
Decoded sentence: -AA1-L-S-EH0-TH-

-
Input sentence: AAMODT
Decoded sentence: -AA1-M-AO0-D-

-
Input sentence: AANCOR
Decoded sentence: -AA1-N-K-AO2-R

-
Input sentence: ABELA
Decoded sentence: -AA0-B-EH1-L-AH0-

-
Input sentence: ABELARD
Decoded sentence: -AE1-B-IH0-L-ER0-D-

-
Input sentence: ABELE
Decoded sentence: -AH0-B-EH1-L-

-
Input sentence: ABELES
Decoded sentence: -AH0-B-EH1-L-Z-

-
Input sentence: ABELES(1)
Decoded sentence: -EY1-B-AH0-L-IY2-Z-

-
Input sentence: ABELL
Decoded sentence: -EY1-B-AH0-L-

-
Input sentence: ABELLA
Decoded sentence: -AH0-B-EH1-L-AH0-

-
Input sentence: ABELN
Decoded sentence: -AE1-B-IH0-L-N-

-
Input sentence: ABELOW
Decoded sentence: -AE1-B-AH0-L-OW0-

-
Input sentence: ABELS
Decoded sentence: -EY1-B-AH0-L-Z-

-
Input sentence: ABELSON
Decoded sentence: -AE1-B-IH0-L-S-AH0-N-

-
Input sentence: ABEND
Decoded sentence: -AE1-B-EH0-N-D-

-
Input sentence: ABEND(1)
Decoded sentence: -AH0-B-EH1-N-D-

-
Input sentence: ABENDROTH
Decoded sentence: -AE1-B-IH0-N-D-R-AO0-TH-

-
Input sentence: ABER
Decoded sentence: -EY1-B-ER0-

-
Input sentence: ABERCROMBIE
Decoded sentence: -AE2-B-ER0-K-R-AA1-M-B-IY0-



-
Input sentence: ABORTIONS
Decoded sentence: -AH0-B-AO1-R-SH-AH0-N-Z-

-
Input sentence: ABORTIVE
Decoded sentence: -AH0-B-AO1-R-T-IH0-V-

-
Input sentence: ABORTS
Decoded sentence: -AH0-B-AO1-R-T-S-

-
Input sentence: ABOTT
Decoded sentence: -AH0-B-AA1-T-

-
Input sentence: ABOU
Decoded sentence: -AH0-B-UW1-

-
Input sentence: ABOUD
Decoded sentence: -AA0-B-UW1-D-

-
Input sentence: ABOUHALIMA
Decoded sentence: -AA2-B-UW0-HH-AA0-L-IY1-M-AH0-

-
Input sentence: ABOUHALIMA'S
Decoded sentence: -AA2-B-UW0-HH-AA0-L-IY1-M-AH0-Z-

-
Input sentence: ABOUND
Decoded sentence: -AH0-B-AW1-N-D-

-
Input sentence: ABOUNDED
Decoded sentence: -AH0-B-AW1-N-D-IH0-D-

-
Input sentence: ABOUNDING
Decoded sentence: -AH0-B-AW1-N-D-IH0-NG-

-
Input sentence: ABOUNDS
Decoded sentence: -AH0-B-AW1-N-D-Z-

-
Input sentence: ABOUT
Decoded sentence: -AH0-B-AW1-T-

-
Input sentence: ABOUT'S
Decoded sentence: -AH0-B-AW1-T-S-

-
Input sentence: ABOVE
Decoded sentence: -AH0-B-AH1-V-

-
Input sentence: ABOVE'S
Decode

-
Input sentence: ABSTRACTION
Decoded sentence: -AE0-B-S-T-R-AE1-K-SH-AH0-N-

-
Input sentence: ABSTRACTIONS
Decoded sentence: -AE0-B-S-T-R-AE1-K-SH-AH0-N-Z-

-
Input sentence: ABSTRACTS
Decoded sentence: -AE1-B-S-T-R-AE0-K-T-S-

-
Input sentence: ABSTRUSE
Decoded sentence: -AH0-B-S-T-R-UW1-S-

-
Input sentence: ABSURD
Decoded sentence: -AH0-B-S-ER1-D-

-
Input sentence: ABSURDIST
Decoded sentence: -AH0-B-S-ER1-D-IH0-S-T-

-
Input sentence: ABSURDITIES
Decoded sentence: -AH0-B-S-ER1-D-AH0-T-IY0-Z-

-
Input sentence: ABSURDITY
Decoded sentence: -AH0-B-S-ER1-D-AH0-T-IY0-

-
Input sentence: ABSURDLY
Decoded sentence: -AH0-B-S-ER1-D-L-IY0-

-
Input sentence: ABT
Decoded sentence: -AE1-B-T-

-
Input sentence: ABT(1)
Decoded sentence: -EY1-B-IY1-T-IY1-

-
Input sentence: ABTS
Decoded sentence: -AE1-B-T-S-

-
Input sentence: ABTS(1)
Decoded sentence: -EY1-B-IY1-T-IY1-Z-

-
Input sentence: ABTS(2)
Decoded sentence: -AE1-B-AH0-L-Z-

-
Input sentence: ABU
Decoded sentence: -AE1-B-UW0-

-
Input s

-
Input sentence: ACCIDENT
Decoded sentence: -AE1-K-S-AH0-D-AH0-N-T-

-
Input sentence: ACCIDENT'S
Decoded sentence: -AE1-K-S-AH0-D-AH0-N-T-S-

-
Input sentence: ACCIDENTAL
Decoded sentence: -AE2-K-S-AH0-D-EH1-N-T-AH0-L-

-
Input sentence: ACCIDENTAL(1)
Decoded sentence: -AE2-K-S-AH0-D-EH1-N-AH0-L-

-
Input sentence: ACCIDENTALLY
Decoded sentence: -AE2-K-S-AH0-D-EH1-N-T-AH0-L-IY0-

-
Input sentence: ACCIDENTALLY(1)
Decoded sentence: -AE2-K-S-AH0-D-EH1-N-AH0-L-IY0-

-
Input sentence: ACCIDENTLY
Decoded sentence: -AE1-K-S-AH0-D-AH0-N-T-L-IY0-

-
Input sentence: ACCIDENTS
Decoded sentence: -AE1-K-S-AH0-D-AH0-N-T-S-

-
Input sentence: ACCION
Decoded sentence: -AE1-CH-IY0-AH0-N-

-
Input sentence: ACCIVAL
Decoded sentence: -AE1-S-IH0-V-AA2-L-

-
Input sentence: ACCLAIM
Decoded sentence: -AH0-K-L-EY1-M-

-
Input sentence: ACCLAIM'S
Decoded sentence: -AH0-K-L-EY1-M-Z-

-
Input sentence: ACCLAIMED
Decoded sentence: -AH0-K-L-EY1-M-D-

-
Input sentence: ACCLAIMING
Decoded sentence: -AH0-K-L-EY1-

-
Input sentence: ACCUSATORY
Decoded sentence: -AH0-K-Y-UW1-Z-AH0-T-AO2-R-IY0-

-
Input sentence: ACCUSE
Decoded sentence: -AH0-K-Y-UW1-Z-

-
Input sentence: ACCUSED
Decoded sentence: -AH0-K-Y-UW1-Z-D-

-
Input sentence: ACCUSER
Decoded sentence: -AH0-K-Y-UW1-Z-ER0-

-
Input sentence: ACCUSERS
Decoded sentence: -AH0-K-Y-UW1-Z-ER0-Z-

-
Input sentence: ACCUSES
Decoded sentence: -AH0-K-Y-UW1-Z-IH0-Z-

-
Input sentence: ACCUSING
Decoded sentence: -AH0-K-Y-UW1-Z-IH0-NG-

-
Input sentence: ACCUSINGLY
Decoded sentence: -AH0-K-Y-UW1-Z-IH0-NG-L-IY0-

-
Input sentence: ACCUSTOM
Decoded sentence: -AH0-K-AH1-S-T-AH0-M-

-
Input sentence: ACCUSTOMED
Decoded sentence: -AH0-K-AH1-S-T-AH0-M-D-

-
Input sentence: ACCUTANE
Decoded sentence: -AE1-K-Y-UW0-T-EY2-N-

-
Input sentence: ACE
Decoded sentence: -EY1-S-

-
Input sentence: ACECOMM
Decoded sentence: -EY1-S-K-AA0-M-

-
Input sentence: ACED
Decoded sentence: -EY1-S-T-

-
Input sentence: ACER
Decoded sentence: -EY1-S-ER0-

-
Input sentence: ACERBIC
D

-
Input sentence: ACORNS
Decoded sentence: -EY1-K-AO0-R-N-Z-

-
Input sentence: ACOSTA
Decoded sentence: -AH0-K-AO1-S-T-AH0-

-
Input sentence: ACOTT
Decoded sentence: -EY1-K-AO0-T-

-
Input sentence: ACOUSTIC
Decoded sentence: -AH0-K-UW1-S-T-IH0-K-

-
Input sentence: ACOUSTICAL
Decoded sentence: -AH0-K-UW1-S-T-IH0-K-AH0-L-

-
Input sentence: ACOUSTICALLY
Decoded sentence: -AH0-K-UW1-S-T-IH0-K-L-IY0-

-
Input sentence: ACOUSTICS
Decoded sentence: -AH0-K-UW1-S-T-IH0-K-S-

-
Input sentence: ACQUAINT
Decoded sentence: -AH0-K-W-EY1-N-T-

-
Input sentence: ACQUAINTANCE
Decoded sentence: -AH0-K-W-EY1-N-T-AH0-N-S-

-
Input sentence: ACQUAINTANCES
Decoded sentence: -AH0-K-W-EY1-N-T-AH0-N-S-IH0-Z-

-
Input sentence: ACQUAINTANCESHIP
Decoded sentence: -AH0-K-W-EY1-N-T-AH0-N-S-SH-IH0-P-

-
Input sentence: ACQUAINTED
Decoded sentence: -AH0-K-W-EY1-N-T-IH0-D-

-
Input sentence: ACQUAINTED(1)
Decoded sentence: -AH0-K-W-EY1-N-IH0-D-

-
Input sentence: ACQUAVIVA
Decoded sentence: -AA0-K-W-AA0-V-IY1-V-

Input sentence: ACUTENESS
Decoded sentence: -AH0-K-Y-UW1-T-N-AH0-S-

-
Input sentence: ACYCLOVIR
Decoded sentence: -AH0-S-IH1-K-L-OW0-V-IH2-R-

-
Input sentence: AD
Decoded sentence: -AE1-D-

-
Input sentence: AD'S
Decoded sentence: -AE1-D-Z-

-
Input sentence: AD-HOC
Decoded sentence: -AE1-D-HH-AA1-K-

-
Input sentence: AD-LIB
Decoded sentence: -AE1-D-L-IH1-B-

-
Input sentence: AD-NAUSEAM
Decoded sentence: -AE1-D-N-AO1-Z-IY0-AA0-M-

-
Input sentence: AD-NAUSEUM
Decoded sentence: -AE1-D-N-AO1-Z-IY0-AH0-M-

-
Input sentence: ADA
Decoded sentence: -EY1-D-AH0-

-
Input sentence: ADA'S
Decoded sentence: -EY1-D-AH0-Z-

-
Input sentence: ADABEL
Decoded sentence: -AE1-D-AH0-B-EH0-L-

-
Input sentence: ADABELLE
Decoded sentence: -AE1-D-AH0-B-AH0-L-

-
Input sentence: ADACHI
Decoded sentence: -AA0-D-AA1-K-IY0-

-
Input sentence: ADAGE
Decoded sentence: -AE1-D-AH0-JH-

-
Input sentence: ADAGIO
Decoded sentence: -AH0-D-AA1-ZH-IY0-OW2-

-
Input sentence: ADAH
Decoded sentence: -AE1-D-AA0-

-
Inpu